<a href="https://colab.research.google.com/github/eoinleen/Biophysics-general/blob/main/structure-visualisation-attempt1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
# Install necessary packages
!pip install py3Dmol ipywidgets biopython

# Import necessary modules
import py3Dmol
import ipywidgets as widgets
from IPython.display import display
import glob
from Bio import PDB
from io import StringIO

# Function to align structures based on Chain A
def align_structures(pdb_files):
    parser = PDB.PDBParser(QUIET=True)
    structures = [parser.get_structure(f'structure_{i}', pdb_file) for i, pdb_file in enumerate(pdb_files)]

    # Align all structures to the first structure that contains Chain A
    ref_structure = None
    for structure in structures:
        if 'A' in structure:
            ref_structure = structure
            break

    if not ref_structure:
        raise ValueError("None of the PDB files contain Chain A.")

    ref_atoms = [atom for atom in ref_structure['A'].get_atoms()]
    for structure in structures:
        if 'A' in structure:
            moving_atoms = [atom for atom in structure['A'].get_atoms()]
            super_imposer = PDB.Superimposer()
            super_imposer.set_atoms(ref_atoms, moving_atoms)
            super_imposer.apply(structure.get_atoms())

    return structures

# Function to get PDB format string from structure
def get_pdb_string(structure):
    pdb_io = PDB.PDBIO()
    pdb_io.set_structure(structure)
    string_io = StringIO()
    pdb_io.save(string_io)
    return string_io.getvalue()

# Function to show the PDB structure
def show_pdb(structures, color="chains_A_green_B_cyan"):
    view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')

    chain_colors = {
        'A': {'lDDT': 'green', 'rainbow': 'spectrum', 'chains_A_green_B_cyan': 'green'},
        'B': {'lDDT': 'cyan', 'rainbow': 'spectrum', 'chains_A_green_B_cyan': 'cyan'},
        'C': {'lDDT': 'orange', 'rainbow': 'spectrum', 'chains_A_green_B_cyan': 'orange'},
        'D': {'lDDT': 'grey70', 'rainbow': 'spectrum', 'chains_A_green_B_cyan': 'grey70'}
    }

    for i, structure in enumerate(structures):
        pdb_str = get_pdb_string(structure)
        view.addModel(pdb_str, 'pdb')
        for chain in structure.get_chains():
            chain_id = chain.id
            if chain_id in chain_colors:
                color_scheme = chain_colors[chain_id].get(color, 'white')
                view.setStyle({'chain': chain_id}, {'cartoon': {'color': color_scheme}})

    view.zoomTo()
    return view

# Function to handle PDB file(s) input and display the structure
def display_structure(pdb_path, color="chains_A_green_B_cyan"):
    parser = PDB.PDBParser(QUIET=True)

    if pdb_path.endswith(".pdb"):
        # Single PDB file
        structure = parser.get_structure('structure_0', pdb_path)
        return show_pdb([structure], color)
    else:
        # Directory with multiple PDB files
        pdb_files = glob.glob(f"{pdb_path}/*.pdb")
        aligned_structures = align_structures(pdb_files)
        return show_pdb(aligned_structures, color)

# Provide the path to your PDB file or directory
pdb_path = '/content/drive/MyDrive/PDB-files/4ash.pdb'  # Update this to your PDB file or directory

# Define interactive widgets
color_widget = widgets.RadioButtons(options=['lDDT', 'rainbow', 'chains_A_green_B_cyan'],
                                    value='chains_A_green_B_cyan', description='Color')

# Create a function to update the view based on widget values
def update_view(color):
    view = display_structure(pdb_path, color)
    view.show()

# Set up interactive display
interactive_plot = widgets.interactive(update_view, color=color_widget)
display(interactive_plot)


interactive(children=(RadioButtons(description='Color', index=2, options=('lDDT', 'rainbow', 'chains_A_green_B…

In [ ]:
from google.colab import drive
drive.mount('/content/drive')